# Deep Q Learning With PyTorch

This notebook should be used as a guide to understand the implementation of DQL algorithm, tha was written in the Agent class. It started as a basic extraction of code comments. Also, we focus here on the 'learn' function, that calculates the loss from a batch of transitions.

<b>Note for developers:</b> Any changes to the agent's code / train function - if approved - should include changes to this notebook. PyTorch is a great framework, but it is a challenge to understand the more complex code fragments, without some wider explanation from the author. 

## Setup

We declare all important stuff first. Net and Memory class:

In [65]:
import torch
import numpy as np
import random
from collections import deque
from torch import autograd, optim, nn
from torch.autograd import Variable
import torch.nn.functional as F

class Net(torch.nn.Module):
    """Neural Network with variable layer sizes and 2 hidden layers."""

    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden1_size)
        self.fc2 = torch.nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = torch.nn.Linear(hidden2_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
    
class Memory(deque):
    """Subclass of deque, storing transitions batches for Agent"""

    def __init__(self, maxlen):
        super().__init__(maxlen=maxlen)
    

Now initialize the networks and the memory (with some random stuff)

In [66]:
initial_state = env.state
actions = env.action_space

input_size = len(initial_state)
hidden1_size = 20
hidden2_size = 10
output_size = len(actions)

q_network = Net(input_size, hidden1_size, hidden2_size, output_size)
target_network = Net(input_size, hidden1_size, hidden2_size, output_size)

memory = Memory(100)
for _ in range(100):
    memory.append((np.random.rand(10), 1, -5, np.random.rand(10), False))

gamma = 0.9
epsilon = 0.1
epsilon_decay = 0.999
epsilon_min = 0.001
batch_size = 4
l_rate = 0.01
optimizer = optim.Adagrad(q_network.parameters(), lr=l_rate)

<b>NOTE: We register q_network.parameters() to the optimizer as we want to update them and nothing else.</b>

Define a function that agent uses to generate batches from memory

In [67]:
def get_experience_batch():
    """
    Retrieves a random batch of transitions from memory and transforms it
    to separate PyTorch Variables.

    Transition is a tuple in form of:
    (state, action, reward, next_state, terminal_state)
    Returns:
        exp_batch - list of Variables in given order:
            [0] - input_state_batch
            [1] - action_batch
            [2] - reward_batch
            [3] - next_state_batch
            [4] - terminal_mask_batch
    """

    exp_batch = [0, 0, 0, 0, 0]
    transition_batch = random.sample(memory, batch_size)

    # Float Tensors
    for i in [0, 2, 3, 4]:
        exp_batch[i] = Variable(torch.Tensor([x[i] for x in transition_batch]))

    # Long Tensor for actions
    exp_batch[1] = Variable(torch.LongTensor([int(x[1]) for x in transition_batch]))

    return exp_batch    

## Training iteration

Now lets break up the *_learn()* method code into smaller chunks and observe, with commentary, what happens.

First, let's see the whole, working function:

In [68]:
def _train():
    if len(memory) > batch_size:

        exp_batch = get_experience_batch()

        input_state_batch = exp_batch[0]
        action_batch = exp_batch[1]
        reward_batch = exp_batch[2]
        next_state_batch = exp_batch[3]
        terminal_mask_batch = exp_batch[4]

        all_q_values = self.q_network(input_state_batch)

        q_values = all_q_values.gather(1, action_batch.unsqueeze(1)).squeeze()

        q_next_max = self.q_network(next_state_batch)
        q_next_max = Variable(q_next_max.data)
        q_next_max, _ = q_next_max.max(dim=1)

        q_t1_max_with_terminal = q_next_max.mul(1 - terminal_mask_batch)

        targets = reward_batch + self.gamma * q_t1_max_with_terminal

        self.optimizer.zero_grad()
        loss = nn.modules.SmoothL1Loss()(q_values, targets)
        loss.backward()
        self.optimizer.step()

Now, lets go step by step. Please note that the sizes of printed Variables are pretty important.

- We sample random transition batch and transform it into separate batches (of autograd.Variable type). This is just a basic data preparation and the logic is handled by <b>get_experience_batch</b> method.
- Make sure you understand how the data looks after this. We had transitions, but now we have separate batches. So, to retrieve the first tranisiton from the batches, you should take the first elements from each of them.

In [69]:
exp_batch = get_experience_batch()

input_state_batch = exp_batch[0]
action_batch = exp_batch[1]
reward_batch = exp_batch[2]
next_state_batch = exp_batch[3]
terminal_mask_batch = exp_batch[4]

print(input_state_batch)
print("\n", action_batch)
print("\n", reward_batch)
print("\n", next_state_batch)
print("\n", terminal_mask_batch)


Variable containing:
 0.4393  0.3313  0.3760  0.1748  0.8239  0.8780  0.5023  0.3082  0.4402  0.3039
 0.8808  0.4934  0.5943  0.0300  0.3950  0.3696  0.3727  0.9839  0.7707  0.0806
 0.7093  0.3182  0.9944  0.9340  0.6316  0.4987  0.4417  0.0009  0.9415  0.6087
 0.6867  0.4861  0.3319  0.7815  0.3298  0.1584  0.8497  0.4971  0.5337  0.5132
[torch.FloatTensor of size 4x10]


 Variable containing:
 1
 1
 1
 1
[torch.LongTensor of size 4]


 Variable containing:
-5
-5
-5
-5
[torch.FloatTensor of size 4]


 Variable containing:
 0.8113  0.1527  0.7908  0.3781  0.1950  0.6430  0.2460  0.5829  0.4210  0.6513
 0.1523  0.0812  0.0383  0.8162  0.2590  0.1824  0.4463  0.2484  0.4993  0.7634
 0.7250  0.2762  0.0536  0.2136  0.4889  0.0851  0.0691  0.6098  0.4505  0.5529
 0.0389  0.0098  0.9415  0.3856  0.3839  0.6806  0.3650  0.8824  0.2243  0.0564
[torch.FloatTensor of size 4x10]


 Variable containing:
 0
 0
 0
 0
[torch.FloatTensor of size 4]



### 1. We start to calculate all the important values needed to compute the error used to train the network.

- As Deep Q Learning states, we want to calculate the error like this: <b>Q(s,a) - (r + max{a}{Q(s_next,a)})</b>

- ( This differs a bit if we use Double Deep Q Learning, so if you work on it, make sure to change it! )

- So lets start with calculating the Q(s) values for each input state in input_state_batch:


In [70]:
all_q_values = q_network(input_state_batch)
print(all_q_values, all_q_values.shape)

Variable containing:
-0.1443 -0.1086 -0.2017 -0.2203
-0.1597 -0.1120 -0.1660 -0.2246
-0.1552 -0.1260 -0.1731 -0.2242
-0.1414 -0.0773 -0.1912 -0.2129
[torch.FloatTensor of size 4x4]
 torch.Size([4, 4])


### 2. Now, retrieve q_values only for actions that were taken - we want Q(s,a), not Q(s)! 

- The squeeze / unsqueeze functions are used because of size mismatches. See http://pytorch.org/docs/stable/torch.html#torch.squeeze

In [71]:
q_values = all_q_values. \
                gather(1, action_batch.unsqueeze(1)).squeeze()
print(q_values)

Variable containing:
-0.1086
-0.1120
-0.1260
-0.0773
[torch.FloatTensor of size 4]



- This use of gather (http://pytorch.org/docs/stable/torch.html#torch.gather) function works the same as a basic loop:

In [72]:
q_values_test = []
for i in range(len(all_q_values)): 
    q_values_test.append(all_q_values[i][action_batch[i]])
    
print(q_values_test)

[Variable containing:
-0.1086
[torch.FloatTensor of size 1]
, Variable containing:
-0.1120
[torch.FloatTensor of size 1]
, Variable containing:
-0.1260
[torch.FloatTensor of size 1]
, Variable containing:
1.00000e-02 *
 -7.7259
[torch.FloatTensor of size 1]
]


- <b>but we cannot use such loop, because Variables are immutable, so they don't have 'append' function etc.</b>

### 3. Calculate q_next_max = max{a}{Q(s_next,a)}

- Max function return the values <b>and</b> indices (http://pytorch.org/docs/stable/torch.html#torch.max), but we just want the values

In [73]:
q_next_max = q_network(next_state_batch)
q_next_max = Variable(q_next_max.data)
print(q_next_max)
q_next_max, _ = q_next_max.max(dim=1)
print(q_next_max)

Variable containing:
-0.1351 -0.1182 -0.1916 -0.2275
-0.1100 -0.0754 -0.2378 -0.2160
-0.1158 -0.0998 -0.2051 -0.2263
-0.1435 -0.1067 -0.1973 -0.2218
[torch.FloatTensor of size 4x4]

Variable containing:
-0.1182
-0.0754
-0.0998
-0.1067
[torch.FloatTensor of size 4]



- So as we see - we calculate the outputs and take the biggest ones
- Note: We create new Variable after the first line. Why?
- We used the q_network parameters to calculate q_next_max, but we don't want the backward() function to propagate twice into these parameters. Creating new Variable 'cuts' this part of computational graph - prevents it.

### 4. If the next state was terminal, we don't calculate the q value the target should be just = r

In [74]:
q_t1_max_with_terminal = q_next_max.mul(1 - terminal_mask_batch)
print(q_t1_max_with_terminal)

Variable containing:
-0.1182
-0.0754
-0.0998
-0.1067
[torch.FloatTensor of size 4]



- So if the next state would be terminal, there would be zeros in corresponding places!

### 5. Calculate the target = r + max{a}{Q(s_next,a)}

- We have the rewards and the max Q values for next states, so we can calculate the target by adding them up.
- Of course, we multiply by gamma here, as stated in the Q-Learning algorithm.

In [75]:
targets = reward_batch + gamma * q_t1_max_with_terminal

print(targets)
print(q_values)

Variable containing:
-5.1064
-5.0678
-5.0898
-5.0960
[torch.FloatTensor of size 4]

Variable containing:
-0.1086
-0.1120
-0.1260
-0.0773
[torch.FloatTensor of size 4]



- So, what we see? We have the networks predictions, and the target values. Nicely trained agent (with reasonable environment transitions) makes the predicitions close to the targets. Our agent does a bad job.

### Loss function and backward()

- To train him into a better direction, we now calculate the loss function (yes, we usually don't use the raw error and calculate a loss function):

In [76]:
optimizer.zero_grad()
loss = nn.modules.SmoothL1Loss()(q_values, targets)
print(loss)
loss.backward()
optimizer.step()

Variable containing:
 4.4841
[torch.FloatTensor of size 1]



- We make use of pytorch's optimizer. We have to zero the gradients before calculating new ones.

- We calculate the loss using built-in SmoothL1Loss, called Huber Loss, which is a recommended choice for Q-Learning

- We call the <b>backward</b> function, which traces all values that were used in the process of calculating the loss. The most important thing here, is that it will calculate the gradients for all q_network weights and biases.

- We calculated the gradients, but the update happens only with <b>optimizer.step()</b> call. It takes  the gradients and updates the parameters (just the parameters that were registered to the optimizer!), finalizing the training iteration. 